## Use the Notebook to build the code to scrape the following Wikipedia page

import pandas as pd
import numpy as np
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [137]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
response = requests.get(url)
#response.text

In [138]:
soup = BeautifulSoup(response.text, "html.parser")

In [139]:
table = soup.find_all('table', class_ = 'wikitable sortable' )
#print(stat_table)
    

#### Scrape table and convert it into a datafram


In [140]:
table = soup.find_all('table')
df = pd.read_html(str(table))[0]
df.head()


,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [141]:
df = df[df.Borough != 'Not assigned']
df = df.rename(columns={'Postal code': 'PostalCode'})
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [142]:
#this does not make any sense, the website already aggregated the data. I dont need to do it! 
df = df.groupby(['PostalCode', 'Borough']).agg(', '.join)
df = df.reset_index()

#because the website already aggregated the data they added a / into it. I need to replace it with a , 
for col in df:
    df['Neighborhood'] = df['Neighborhood'].str.replace(' /',', ')
    
df.head(10)
#.agg(lambda column: ",".join(column)).reindex(columns=df.columns)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [143]:
df.loc[df['Neighborhood']=='Not assigned', ['Neighborhood']] 
df.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


#### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [144]:
df.shape

(103, 3)